Notebook implementing Cox, Ross and Rubinstein Binomial Tree. 

Tested against EUROPEAN and AMERICAN options. 

# Binomial Trees

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from option_param import Parameters


In [23]:
param = Parameters(100,100,0,1,0.05,0.0,0.2)
param.get_attribute()

stock     : 100.0
strike    : 100.0
t         : 0.0
T         : 1.0
tau       : 1.0
rate      : 0.05
dividend  : 0.0
vol       : 0.2


## CRR: Cox, Ross, and Rubinstein (1979): 

In [24]:
class BinomialTree:
    """
    Class Binomial tree. 
    By default use CRR methodology

    
    --------------------------------
    Note: 
        if Volatility paramter is given, u and d are calculated
        automatically. 
        if instead parameter u and d are given,
        you can upload directly into the object as:
        obj = Tree(param, n)
        obj.u = 1.2
        obj.d = 1/1.2
        
    Future Idea: 
    To use the Binomial Asset Pricing Model as per Steven E. Shreve
    book's Stochastic Calculos for Finance I, use obj.set_apm(u, d),
    where u and d have to be manually inputed. 
    """
    
    def __init__(self, param, n):
        self.param = param      # Parameter object
        self.n = n              # number of timestep
        
        self.dt = self.param.tau / self.n
        
        self.u = 0 # up movement
        self.d = 0 # down movement

        self.p = 0 # probability 
        
        
        self.t_stock = np.zeros((n+1,n+1))
      
        self.set_crr() # By default using the CRR model
                
    def set_crr(self):
        """
        Set parameter according to the Cox, Ross and Rubinstein (1979) model
        """
        
        self.u = np.exp(self.param.vol * np.sqrt(self.dt)) # up
        self.d = 1/self.u # down (faster)
        
        self.p = (np.exp(self.param.rate * self.dt) - self.d) / (self.u - self.d)

    def set_apm(self, u, d):
        """
        Set parameter according the The Binomial Asset Pricing Model
        Steven E. Schreve
        Stochastic Calculus for Finance 1: The Binomial Asset Pricing Model 
        """
        self.u = u
        self.d = d
        self.p = (1 + self.param.rate - self.d) / (self.u - self.d)
            
    def set_tree(self):
        """
        Summary: 
            Calculate Binomial Tree using CRR method
        Comment:
            - Faster version. Instead of using a double loop, 
              you can vectorise one of the loop
            - For european option pricing,
              only Terminal Stock price required. 
        """
        for i in range(self.n+1):
            self.t_stock[:i+1,i:i+1] = (self.param.stock * self.u**(np.arange(i,-1,-1)) \
                * self.d**(np.arange(0,i+1,1))).reshape(-1,1)
            
    
    def set_european(self):

        self.t_stock[:,-1] = (self.param.stock * self.u**(np.arange(self.n, -1, -1)) \
                * self.d**(np.arange(0,self.n + 1, 1)))

        self.t_eu_c = self.t_stock.copy()
        self.t_eu_p = self.t_stock.copy()
        
        #Intrinsic option price at the final node
        self.t_eu_c[:,-1] =  np.maximum(self.t_eu_c[:,-1] \
            - self.param.strike, 0.0) # call

        self.t_eu_p[:,-1] =  np.maximum(self.param.strike \
            - self.t_eu_p[:,-1], 0.0) # put

        # Backward tree:
        for i in range(self.n,0,-1):
            # Call
            self.t_eu_c[:i, i-1] = np.exp(-self.param.rate * self.dt) \
                * (self.p * self.t_eu_c[:i, i] \
                + (1 - self.p) * self.t_eu_c[1:i+1, i])

            # Put
            self.t_eu_p[:i, i-1] = np.exp(-self.param.rate * self.dt) \
                * (self.p * self.t_eu_p[:i, i] \
                + (1 - self.p) * self.t_eu_p[1:i+1, i])


    def set_american(self):

        self.set_tree()

        self.t_am_c = self.t_stock.copy()
        self.t_am_p = self.t_stock.copy()

        self.t_am_c[:,-1] = np.maximum(self.t_stock[:,-1] \
            - self.param.strike,0.0)

        self.t_am_p[:,-1] = np.maximum(self.param.strike \
            - self.t_stock[:,-1], 0.0)

        for i in range(self.n-1,-1,-1):

            # Call
            ex_c = self.t_stock[:,i] - self.param.strike
            wait_c = np.exp(-self.param.rate * self.dt) \
                * (self.p * self.t_am_c[:i+1, i+1] \
                + (1 - self.p) * self.t_am_c[1:i+2, i+1])

            self.t_am_c[:i+1,i] = np.maximum(ex_c[:i+1], wait_c[:i+1])

            # Put
            ex_p =  self.param.strike - self.t_stock[:,i]
            wait_p = np.exp(-self.param.rate * self.dt) \
                * (self.p * self.t_am_p[:i+1, i+1] \
                + (1 - self.p) * self.t_am_p[1:i+2, i+1])

            self.t_am_p[:i+1,i] = np.maximum(ex_p[:i+1], wait_p[:i+1])     
               

### Function

Check algorithm: (works!)
- European put: https://www.youtube.com/watch?v=gOStLKAehjA
- American put: https://www.youtube.com/watch?v=nr8Rw8RYSdU

In [25]:
test = Parameters(102, 100, 0, 0.5, 0.02, 0.0, 0.2000773189775976)
t = BinomialTree(test, 4)
t.set_crr()
t.u = 1.0733 # set u and d manually
t.d = 0.9317
t.set_european()
t.set_american()

In [26]:
pd.DataFrame(t.t_stock)

,0,1,2,3,4
0,102.0,109.4766,117.501235,126.114075,135.358237
1,0.0,95.0334,101.999348,109.475900,117.500484
2,0.0,0.0000,88.542619,95.032793,101.998696
3,0.0,0.0000,0.000000,82.495158,88.542053
4,0.0,0.0000,0.000000,0.000000,76.860739


In [27]:
t.t_eu_c

array([[ 7.26157935, 11.64435194, 17.99928905, 26.36338857, 35.35823701],
       [ 0.        ,  2.91514148,  5.34768248,  9.72526313, 17.50048395],
       [ 0.        ,  0.        ,  0.49718409,  0.9968551 ,  1.99869644],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])

In [28]:
pd.DataFrame(t.t_eu_p)

,0,1,2,3,4
0,4.267774,1.421533,0.000000,0.000000,0.000000
1,0.000000,7.135394,2.850188,0.000000,0.000000
2,0.000000,0.000000,11.456339,5.714657,0.000000
3,0.000000,0.000000,0.000000,17.255399,11.457947
4,0.000000,0.000000,0.000000,0.000000,23.139261


In [29]:
t.t_am_c

array([[ 7.26157935, 11.64435194, 17.99928905, 26.36338857, 35.35823701],
       [ 0.        ,  2.91514148,  5.34768248,  9.72526313, 17.50048395],
       [ 0.        ,  0.        ,  0.49718409,  0.9968551 ,  1.99869644],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])

In [30]:
t.t_am_p

array([[ 4.29872139,  1.42153265,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  7.19744294,  2.85018792,  0.        ,  0.        ],
       [ 0.        ,  0.        , 11.58074878,  5.71465679,  0.        ],
       [ 0.        ,  0.        ,  0.        , 17.50484208, 11.45794701],
       [ 0.        ,  0.        ,  0.        ,  0.        , 23.13926137]])

In [31]:
# Example from John C. Hull 
# Options, Futures, and Other Derivatives 9th Edition
# p475 - 476 || Example 21.1 - Fig. 21.3
test = Parameters(50, 50, 0, 5/12, 0.1, 0.0, 0.4)
t = BinomialTree(test, 5)
t.set_american()

In [32]:
pd.DataFrame(t.t_am_p)

,0,1,2,3,4,5
0,4.488459,2.162519,0.635984,0.000000,0.000000,0.000000
1,0.000000,6.959743,3.771142,1.301666,0.000000,0.000000
2,0.000000,0.000000,10.361294,6.378043,2.664116,0.000000
3,0.000000,0.000000,0.000000,14.638882,10.310650,5.452637
4,0.000000,0.000000,0.000000,0.000000,18.495109,14.638882
5,0.000000,0.000000,0.000000,0.000000,0.000000,21.930804


In [33]:
# Example from John C. Hull 
# Options, Futures, and Other Derivatives 9th Edition
# p475 - 476 || Example 21.1 - Fig. 21.3
test = Parameters(50, 50, 0, 5/12, 0.1, 0.0, 0.4)
t = BinomialTree(test, 8)
t.set_american()
pd.DataFrame(t.t_stock)

,0,1,2,3,4,5,6,7,8
0,50.0,54.779175,60.015160,65.751618,72.036388,78.921877,86.465506,94.730181,103.784823
1,0.0,45.637781,50.000000,54.779175,60.015160,65.751618,72.036388,78.921877,86.465506
2,0.0,0.000000,41.656142,45.637781,50.000000,54.779175,60.015160,65.751618,72.036388
3,0.0,0.000000,0.000000,38.021878,41.656142,45.637781,50.000000,54.779175,60.015160
4,0.0,0.000000,0.000000,0.000000,34.704683,38.021878,41.656142,45.637781,50.000000
5,0.0,0.000000,0.000000,0.000000,0.000000,31.676895,34.704683,38.021878,41.656142
6,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,28.913264,31.676895,34.704683
7,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,26.390744,28.913264
8,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,24.088300
